# Building AI Applications by Customizing DSPy Modules

https://dspy.ai/tutorials/custom_module/

https://github.com/stanfordnlp/dspy/blob/main/docs/docs/tutorials/custom_module/index.ipynb

Let's illustrate this with a practical code example. We will build a simple Retrieval-Augmented Generation (RAG) application with multiple stages:

* Query Generation: Generate a suitable query based on the user's question to retrieve relevant context.
* Context Retrieval: Fetch context using the generated query.
* Answer Generation: Produce a final answer based on the retrieved context and the original question.  

The code implementation for this multi-stage program is shown below.

In [ ]:
import dspy

class QueryGenerator(dspy.Signature):
    """Generate a query based on question to fetch relevant context"""
    question: str = dspy.InputField()
    query: str = dspy.OutputField()

def search_wikipedia(query: str) -> list[str]:
    """Query ColBERT endpoint, which is a knowledge source based on wikipedia data"""
    try:
        results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=1)
        return [x["text"] for x in results]
    except (KeyError, Exception) as e:
        # Fallback to placeholder context if ColBERT endpoint is unavailable
        print(f"Warning: ColBERT endpoint error ({e}). Using placeholder context.")
        print("Note: To fix this, either ensure the ColBERT server is running or configure a different retriever.")
        return [f"Unable to retrieve context for query: '{query}'. The ColBERT endpoint is currently unavailable."]

class RAG(dspy.Module):
    def __init__(self):
        self.query_generator = dspy.Predict(QueryGenerator)
        self.answer_generator = dspy.ChainOfThought("question,context->answer")

    def forward(self, question, **kwargs):
        query = self.query_generator(question=question).query
        context = search_wikipedia(query)[0]
        return self.answer_generator(question=question, context=context).answer


In [11]:
from langfuse import get_client
from openinference.instrumentation.dspy import DSPyInstrumentor

import os

os.environ["LANGFUSE_PUBLIC_KEY"] = os.getenv("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.getenv("LANGFUSE_SECRET_KEY")
os.environ["LANGFUSE_BASE_URL"] = os.getenv("LANGFUSE_HOST")

langfuse = get_client()
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")
# Enable tracing for DSPy
DSPyInstrumentor().instrument()

Attempting to instrument while already instrumented


Langfuse client is authenticated and ready!


In [12]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY", "{your openai key}")
api_base = os.getenv("OPENAI_API_BASE", "https://api.openai.com")
model_id = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

In [13]:
lm = dspy.LM(f'openai/{model_id}', api_key=api_key, api_base=api_base)
dspy.configure(lm=lm, track_usage=True)

rag = RAG()
print(rag(question="Is Lebron James the basketball GOAT?"))

2026/02/01 15:37:24 WARNING dspy.primitives.module: Failed to set LM usage. Please return `dspy.Prediction` object from dspy.Module to enable usage tracking.


Note: To fix this, either ensure the ColBERT server is running or configure a different retriever.
LeBron James is unquestionably one of the greatest basketball players ever and a leading contender for the GOAT title, but whether he is the definitive GOAT depends on the criteria you prioritize—statistics, longevity, versatility, championship record, or cultural impact. The debate remains open, with many fans and analysts still weighing him against other legends, especially Michael Jordan.
